In [1]:
"""
re-writing CFS cross dataset for re-do
modified from: allenadultmouseISH/allbyallCFS.py, allenadultmouseISH/debugCFSallbyall_twentytwo.ipynb, and spatial/10_crossdataset.py
major changes: MWU vectorized, getting and testing feat sets using pd.apply

Shaina Lu
Zador + Gillis Labs
May 2020
"""

'\nre-writing CFS cross dataset for re-do\nmodified from: allbyallCFS.py and debugCFSallbyall_twentytwo.ipynb\nmajor changes:\n\nShaina Lu\nZador + Gillis Labs\nMay 2020\n'

In [97]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split #stratify train/test split
import random
import matplotlib.pyplot as plt
import logging
import timeit

## Read in files and pre-processing

In [3]:
#file paths
ALLEN_FILT_PATH = "/home/slu/spatial/data/ABAISH_filt_v6_avgdup.h5"
ONTOLOGY_PATH = "/data/slu/allen_adult_mouse_ISH/ontologyABA.csv"
ST_CANTIN_FILT_PATH = "/home/slu/spatial/data/cantin_ST_filt_v2.h5"

#outfiles
FEATSETS_OUT = "STtoABA_featsetsSTtrain_f1_CFS_052120.csv"
MOD_TEST_OUT = "STtoABA_STtest_f1_CFS_052120.csv"
MOD_TRAIN_OUT = "STtoABA_STtrain_f1_CFS_052120.csv"
CROSS_ALL_OUT = "STtoABA_ABAall_f1_CFS_052120.csv"

Read functions copied from 10_crossdataset.py

In [4]:
def read_ABAdata():
    """read in all ABA datasets needed using pandas"""
    metabrain = pd.read_hdf(ALLEN_FILT_PATH, key='metabrain', mode='r')
    voxbrain = pd.read_hdf(ALLEN_FILT_PATH, key='avgvoxbrain', mode='r')
    propontvox = pd.read_hdf(ALLEN_FILT_PATH, key='propontology', mode='r')
    #geneIDName = pd.read_hdf(ALLEN_FILT_PATH, key='geneIDName', mode='r')	

    return metabrain, voxbrain, propontvox

def read_STdata():
    """read in all ST datasets needed using pandas"""
    STspotsmeta = pd.read_hdf(ST_CANTIN_FILT_PATH, key='STspotsmeta', mode='r')
    STspots = pd.read_hdf(ST_CANTIN_FILT_PATH, key='STspots', mode='r')
    STpropont = pd.read_hdf(ST_CANTIN_FILT_PATH, key='STpropont', mode='r')
    
    return STspotsmeta, STspots, STpropont

def read_ontology():
    ontology = pd.read_csv(ONTOLOGY_PATH)
    ontology = ontology.drop([ontology.columns[5], ontology.columns[6]], axis=1)
    ontology = ontology.fillna(-1)  #make root's parent -1

    return ontology

Pre-processing functions copied from 10_crossdataset.py

In [5]:
def filterproponto(sampleonto):
    """pre-processing for propogated ontology"""
    #remove brain areas that don't have any samples
    sampleonto_sums = sampleonto.apply(lambda col: col.sum(), axis=0)
    sampleonto = sampleonto.loc[:,sampleonto_sums > 5] #greater than 5 becuase less is not enough for train/test split to have non-zero areas
    
    return sampleonto

def getleaves(propontvox, ontology):
    """helper function to get only leaf brain areas"""
    #leaves are brain areas in the ontology that never show up in the parent column
    allareas = list(propontvox)
    parents = list(ontology.parent)
    for i in range(len(parents)): #convert parents from float to int, ids are ints
        parents[i] = int(parents[i])
    
    #remove parents from all areas
    leaves = []
    for area in allareas:
        if int(area) not in parents:
            leaves.append(area)
    
    print("number of leaf areas: %d" %len(leaves))
    return leaves

def findoverlapareas(STonto, propontvox, ontology):
    """find leaf brain areas overlapping between the two datasets"""
    leafST = getleaves(STonto, ontology)
    leafABA = getleaves(propontvox, ontology)

    leafboth = [] 
    for i in range(len(leafABA)):
        if leafABA[i] in leafST:
            leafboth.append(leafABA[i])
    
    STonto = STonto.loc[:,leafboth]
    propontvox = propontvox.loc[:,leafboth]
    
    return STonto, propontvox    

def zscore(voxbrain):
    """zscore voxbrain or subsets of voxbrain (rows: voxels, cols: genes)"""
    #z-score on whole data set before splitting into test and train
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(voxbrain)
    z_voxbrain = scaler.transform(voxbrain)
    
    #store z-scored voxbrain as pandas dataframe
    z_voxbrain = pd.DataFrame(z_voxbrain)
    z_voxbrain.columns = voxbrain.columns
    z_voxbrain.index = voxbrain.index
    
    return z_voxbrain

def analytical_auroc(featurevector, binarylabels):
    """analytical calculation of auroc
       inputs: feature (mean rank of expression level), binary label (ctxnotctx)
       returns: auroc
    """
    #sort ctxnotctx binary labels by mean rank, aescending
    s = sorted(zip(featurevector, binarylabels))
    feature_sort, binarylabels_sort = map(list, zip(*s))

    #get the sum of the ranks in feature vector corresponding to 1's in binary vector
    sumranks = 0
    for i in range(len(binarylabels_sort)):
        if binarylabels_sort[i] == 1:
            sumranks = sumranks + feature_sort[i]
    
    poslabels = binarylabels.sum()
    neglabels = (len(binarylabels) - poslabels)
    
    auroc = ((sumranks/(neglabels*poslabels)) - ((poslabels+1)/(2*neglabels)))
    
    return auroc

def getoverlapgenes(STspots, ABAvox):
    ABAgenes = list(ABAvox)
    STgenes = list(STspots)
    
    #get overlapping genes
    overlap = []
    for i in range(len(ABAgenes)):
        if ABAgenes[i] in STgenes:
            overlap.append(ABAgenes[i])
    
    print("number of overlapping genes: %d" %len(overlap))
    
    #index datasets to keep only genes that are overlapping
    STspots = STspots.loc[:,overlap]
    ABAvox = ABAvox.loc[:,overlap]
    
    return STspots, ABAvox


### testing vectorized MWU against scipy

In [98]:
%%timeit
#def getDEgenes2(Xtrain, ytrain):  
#modified vectorized MWU from Ben and scipy.stats.mannwhitneyu source code
Xtrain_ranked = Xtrain.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)
n1 = ytrain.sum() #instances of brain area marked as 1
n2 = len(ytrain) - n1
U = Xtrain_ranked.loc[ytrain==1, :].sum() - ((n1*(n1+1))/2)

T = Xtrain_ranked.apply(lambda col: sp.stats.tiecorrect(col), axis=0)
sd = np.sqrt(T * n1 * n2 * (n1+n2+1) / 12.0)
meanrank = n1*n2/2.0 + 0.5
z = (U - meanrank) / sd
#p = distributions.norm.sf(z)

#Z = np.abs(U - ((n1*n2)/2)) / np.sqrt(n1*n2*(n1+n2+1)/12)
pvals = pd.Series(stats.norm.sf(z), index=list(Xtrain))

4.09 s ± 6.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%%timeit
cols = list(Xtrain)
    
#one-sided Mann-Whitney, Ha: areaofinterest < not areaofinterest
u2 = []
pvals2 = []
genes2 = []
errors2 = []
for i in range(len(cols)):
    try:
        curr_u, curr_pval = sp.stats.mannwhitneyu(Xtrain.loc[ytrain ==1,\
                            cols[i]],Xtrain.loc[ytrain == 0,cols[i]],alternative='greater')
        u2.append(curr_u)
        pvals2.append(curr_pval)
        genes2.append(cols[i])
    except:   #some genes raise the error that "all numbers are identical in mwu"
        u2.append(1)
        pvals2.append(1)
        errors2.append(cols[i])

11.5 s ± 2.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
#U stats are not equal because scipy implementation has a correction when all are tied
print((U == u2).sum())
print(U.shape)

11050
(14299,)


In [125]:
#p-vals are actually all equal despite the difference here due to differences in floats
print((pvals == pvals2).sum())
print(pvals.shape)

14255
(14299,)


In [126]:
#few pvals that are different, are just due to slight differences in floats
for i in range(len(np.where(pvals !=pvals2)[0])):
    index = np.where(pvals!=pvals2)[0][i]
    print("vector: %f  |  original: %f" %(pvals.iloc[index], pvals2[index]))

vector: 0.275606  |  original: 0.275606
vector: 0.071765  |  original: 0.071765
vector: 0.275606  |  original: 0.275606
vector: 0.275606  |  original: 0.275606
vector: 0.288289  |  original: 0.288289
vector: 0.009699  |  original: 0.009699
vector: 0.071765  |  original: 0.071765
vector: 0.071765  |  original: 0.071765
vector: 0.275606  |  original: 0.275606
vector: 0.275606  |  original: 0.275606
vector: 0.071765  |  original: 0.071765
vector: 0.009914  |  original: 0.009914
vector: 0.419548  |  original: 0.419548
vector: 0.275606  |  original: 0.275606
vector: 0.071765  |  original: 0.071765
vector: 0.791791  |  original: 0.791791
vector: 0.791791  |  original: 0.791791
vector: 0.275606  |  original: 0.275606
vector: 0.071765  |  original: 0.071765
vector: 0.275606  |  original: 0.275606
vector: 0.071765  |  original: 0.071765
vector: 0.275606  |  original: 0.275606
vector: 0.745346  |  original: 0.745346
vector: 0.008798  |  original: 0.008798
vector: 0.071765  |  original: 0.071765


show that flipping n1 and n2 and setting ytrain == 0 is truely alternative = less

In [ ]:
#my 'vectorized' version
Xtrain_ranked = Xtrain.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)
n2 = ytrain.sum() #instances of brain area marked as 1
n1 = len(ytrain) - n1
U = Xtrain_ranked.loc[ytrain==0, :].sum() - ((n1*(n1+1))/2)

T = Xtrain_ranked.apply(lambda col: sp.stats.tiecorrect(col), axis=0)
sd = np.sqrt(T * n1 * n2 * (n1+n2+1) / 12.0)
meanrank = n1*n2/2.0 + 0.5
z = (U - meanrank) / sd

pvals = pd.Series(stats.norm.sf(z), index=list(Xtrain))

#scipy versoin
cols = list(Xtrain)
    
#one-sided Mann-Whitney, Ha: areaofinterest < not areaofinterest
u2 = []
pvals2 = []
genes2 = []
errors2 = []
for i in range(len(cols)):
    try:
        curr_u, curr_pval = sp.stats.mannwhitneyu(Xtrain.loc[ytrain ==1,\
                            cols[i]],Xtrain.loc[ytrain == 0,cols[i]],alternative='less')
        u2.append(curr_u)
        pvals2.append(curr_pval)
        genes2.append(cols[i])
    except:   #some genes raise the error that "all numbers are identical in mwu"
        u2.append(1)
        pvals2.append(1)
        errors2.append(cols[i])

In [132]:
print((pvals == pvals2).sum())
print(pvals.shape)

for i in range(len(np.where(pvals !=pvals2)[0])):
    index = np.where(pvals!=pvals2)[0][i]
    print("vector: %f  |  original: %f" %(pvals.iloc[index], pvals2[index]))

14283
(14299,)
vector: 0.342334  |  original: 0.342334
vector: 0.745346  |  original: 0.745346
vector: 0.193018  |  original: 0.193018
vector: 0.342334  |  original: 0.342334
vector: 0.041490  |  original: 0.041490
vector: 0.288289  |  original: 0.288289
vector: 0.106370  |  original: 0.106370
vector: 0.293989  |  original: 0.293989
vector: 0.694708  |  original: 0.694708
vector: 0.058077  |  original: 0.058077
vector: 0.154992  |  original: 0.154992
vector: 0.096759  |  original: 0.096759
vector: 0.657666  |  original: 0.657666
vector: 0.106370  |  original: 0.106370
vector: 0.305292  |  original: 0.305292
vector: 0.845008  |  original: 0.845008


In conclusion, the vectorized version of MWU from deconstructing the function takes less than half the time of the non-vectorized version. Some steps are still a bit linear (tiecorrect, ranking, but pd. apply helps with that). \
\
The p-values between the my version and the scipy version are the same. My vectorized version implements alternative = "greater" for x > y. Just flip order of inputs to do alternative = "less."

### CFS functions

Modified from previous for speed-ups

In [247]:
def calcDE(Xtrain, ytrain):
    #Ha: areaofinterest > not areaofinterest; i.e. alternative = greater
    Xtrain_ranked = Xtrain.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)
    n1 = ytrain.sum() #instances of brain area marked as 1
    n2 = len(ytrain) - n1
    U = Xtrain_ranked.loc[ytrain==1, :].sum() - ((n1*(n1+1))/2)

    T = Xtrain_ranked.apply(lambda col: sp.stats.tiecorrect(col), axis=0)
    sd = np.sqrt(T * n1 * n2 * (n1+n2+1) / 12.0)
    meanrank = n1*n2/2.0 + 0.5
    z = (U - meanrank) / sd

    pvals_greater = pd.Series(stats.norm.sf(z), index=list(Xtrain), name='pvals_greater')
    
    #Ha: areaofinterest < notareaofinterest; i.e. alternative = less
    Xtrain_ranked = Xtrain.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)
    n2 = ytrain.sum() #instances of brain area marked as 1
    n1 = len(ytrain) - n1
    U = Xtrain_ranked.loc[ytrain==0, :].sum() - ((n1*(n1+1))/2)

    T = Xtrain_ranked.apply(lambda col: sp.stats.tiecorrect(col), axis=0)
    sd = np.sqrt(T * n1 * n2 * (n1+n2+1) / 12.0)
    meanrank = n1*n2/2.0 + 0.5
    z = (U - meanrank) / sd

    pvals_less = pd.Series(stats.norm.sf(z), index=list(Xtrain), name='pvals_less')
    
    allpvals = pd.concat([pvals_greater, pvals_less], axis=1)
    return allpvals

In [248]:
def getDEgenes(allpvals, numtotal):
    #melt
    allpvals['gene'] = allpvals.index
    allpvals_melt = allpvals.melt(id_vars='gene')
    #sort by p-value
    allpvals_melt = allpvals_melt.sort_values(by='value', ascending=True)
    #get top X number of DE genes
    topDEgenes = allpvals_melt.iloc[0:numtotal, :]
    
    return topDEgenes

In [249]:
def get_featset(featurecorrs, ranksdf, ylabels, seedgene):
    """picking feature set based on fwd sellection, random seed, and lowest possible corr,
       stop when average auroc prediction is no longer improving
       inputs: featurecorrs - correlation matrix of features being considered; seedgene - gene to start CFS
       returns: feature set"""
    #start with passed in randomly picked gene
    featset = [seedgene]
    #get start performance
    curr_auroc = analytical_auroc(sp.stats.mstats.rankdata(ranksdf.loc[:,featset].mean(axis=1)), ylabels)
    improving = True
    while improving:
        #look at all other possible features and take lowest correlated to seed, others in feat set
        means = featurecorrs.loc[:,featset].mean(axis=1)  #get average corr across choosen features
        featset.append(means.idxmin())                  #gets row name of min mean corrs, picks first of ties
        #check featset performance
        new_auroc = analytical_auroc(sp.stats.mstats.rankdata(ranksdf.loc[:,featset].mean(axis=1)),ylabels)
        if new_auroc <= curr_auroc:  #if not improved, stop
            featset.pop(len(featset)-1)
            final_auroc = curr_auroc
            improving = False
        else:
            curr_auroc = new_auroc
            
    return featset

In [254]:
def applyCFS(zXtrain, zXtest, zXcross, ytrain, ytest, ycross):
    #calculate DE for all genes across the two brain areas
    allpvals = calcDE(zXtrain, ytrain)
    #get top X DE genes
    topDEgenes = getDEgenes(allpvals, 500)

    #ranks DE genes
    #train
    rankedXtrain = zXtrain.loc[:, topDEgenes.gene]
    rankedXtrain.loc[:,(topDEgenes.variable=='pvals_less').values] = \
                     -1 * rankedXtrain.loc[:,(topDEgenes.variable=='pvals_less').values]
    rankedXtrain = rankedXtrain.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)
    #test
    rankedXtest = zXtest.loc[:, topDEgenes.gene]
    rankedXtest.loc[:,(topDEgenes.variable=='pvals_less').values] = \
                    -1 * rankedXtest.loc[:,(topDEgenes.variable=='pvals_less').values]
    rankedXtest = rankedXtest.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)
    #cross
    rankedXcross = zXcross.loc[:, topDEgenes.gene]
    rankedXcross.loc[:,(topDEgenes.variable=='pvals_less').values] = \
                    -1 * rankedXcross.loc[:,(topDEgenes.variable=='pvals_less').values]
    rankedXcross = rankedXcross.apply(lambda col: sp.stats.mstats.rankdata(col), axis=0)

    #correlation matrix (spearman, b/c already ranked)
    traincorrs = np.corrcoef(rankedXtrain.values.T)
    traincorrs = pd.DataFrame(traincorrs, index=topDEgenes.gene.values, columns=topDEgenes.gene.values)

    #get 100 feature sets using CFS
    random.seed(0)
    random.seed(42)
    startingpts = pd.Series(random.sample(list(traincorrs),100))
    featsets = startingpts.apply(lambda x: get_featset(traincorrs, rankedXtrain, ytrain, x))
    trainaurocs = featsets.apply(lambda x: analytical_auroc(sp.stats.mstats.rankdata(rankedXtrain.loc[:,x].mean(axis=1)), ytrain))
    testaurocs = featsets.apply(lambda x: analytical_auroc(sp.stats.mstats.rankdata(rankedXtest.loc[:,x].mean(axis=1)), ytest))
    crossaurocs = featsets.apply(lambda x: analytical_auroc(sp.stats.mstats.rankdata(rankedXcross.loc[:,x].mean(axis=1)), ycross))

    #return all 100 feature sets and aurocs
    return featsets, trainaurocs, testaurocs, crossaurocs

In [271]:
#mod_propont = STpropont
#mod_data = STspots
#cross_propont = ABApropont
#cross_data = ABAvox
def getallbyall(mod_data, mod_propont, cross_data, cross_propont):
    #initialize zeros dataframe to store entries
    allbyall_featsets = pd.DataFrame(index=list(mod_propont), columns=list(mod_propont))
    allbyall_test = pd.DataFrame(index=list(mod_propont), columns=list(mod_propont))
    allbyall_train = pd.DataFrame(index=list(mod_propont), columns=list(mod_propont))
    allbyall_cross = pd.DataFrame(index=list(mod_propont), columns=list(mod_propont))

    areas = list(mod_propont)
    #for each column, brain area
    for i in range(mod_propont.shape[1]):
        logging.debug("starting col %d" %i)
        #for each row in each column
        for j in range(i+1,mod_propont.shape[1]): #upper triangular!
            #print("brain area j: %d" %j)
            area1 = areas[i]
            area2 = areas[j]
            #get binary label vectors
            ylabels = mod_propont.loc[mod_propont[area1]+mod_propont[area2] != 0, area1]
            ycross = cross_propont.loc[cross_propont[area1]+cross_propont[area2] != 0, area1]
            #ylabels = pd.Series(np.random.permutation(ylabels1),index=ylabels1.index) #try permuting
            #subset train and test sets for only samples in the two areas
            Xcurr = mod_data.loc[mod_propont[area1]+mod_propont[area2] != 0, :]
            Xcrosscurr = cross_data.loc[cross_propont[area1]+cross_propont[area2] != 0, :]
            #split train test for X data and y labels
            #split data function is seeded so all will split the same way
            Xtrain, Xtest, ytrain, ytest = train_test_split(Xcurr, ylabels, test_size=0.5,\
                                                            random_state=42, shuffle=True,\
                                                            stratify=ylabels)
            #z-score train and test folds
            zXtrain = zscore(Xtrain)
            zXtest = zscore(Xtest)
            zXcross = zscore(Xcrosscurr)

            featsets, currauroc_train, currauroc_test, currauroc_cross = applyCFS(zXtrain, zXtest, zXcross, ytrain, ytest, ycross)
            allbyall_featsets.iloc[i,j] = featsets.values
            allbyall_train.iloc[i,j] = currauroc_train.values
            allbyall_test.iloc[i,j] = currauroc_test.values
            allbyall_cross.iloc[i,j] = currauroc_cross.values

            break

        #periodically save
        #if i%10 == 0:
        #    logging.debug("saving")
        #    allbyall_featsets.to_csv(FEATSETS_OUT, sep=',', header=True, index=False)
        #    allbyall_train.to_csv(ST_TRAIN_OUT, sep=',', header=True, index=False)
        #    allbyall_test.to_csv(ST_TEST_OUT, sep=',', header=True, index=False)
        #    allbyall_cross.to_csv(ABA_ALL_OUT, sep=',', header=True, index=False)

        break


    return allbyall_featsets, allbyall_train, allbyall_test, allbyall_cross

### Main

In [6]:
#read in data
ontology = read_ontology()
ABAmeta, ABAvox, ABApropont = read_ABAdata()
STmeta, STspots, STpropont = read_STdata()

In [7]:
#filter brain areas for those that have at least x samples
STpropont = filterproponto(STpropont)
ABApropont = filterproponto(ABApropont)
#filter brain areas for overlapping leaf areas
STpropont, ABApropont = findoverlapareas(STpropont, ABApropont, ontology)

number of leaf areas: 461
number of leaf areas: 560


In [8]:
#keep only genes that are overlapping between the two datasets
STspots, ABAvox = getoverlapgenes(STspots, ABAvox)

number of overlapping genes: 14299


In [272]:
#predictability matrix using CFS
allbyall_featsets, allbyall_train, allbyall_test, allbyall_cross = getallbyall(STspots, STpropont, ABAvox, ABApropont)

In [ ]:
#write AUROC matrices to outfiles
#allbyall_featsets.to_csv(FEATSETS_OUT, sep=',', header=True, index=False)
#allbyall_train.to_csv(MOD_TRAIN_OUT, sep=',', header=True, index=False)
#allbyall_test.to_csv(MOD_TEST_OUT, sep=',', header=True, index=False)
#allbyall_cross.to_csv(CROSS_ALL_OUT, sep=',', header=True, index=False)